#### [0121 WORK 다변량 분류 모델]
- 주제 : 숫자 분류
- 데이터 : mnist_train.csv
- 구성 : 피쳐 + 타겟
- 학습 : 지도학습  + 분류
- 구현 : 인공신경망

[1] 모듈 로딩 및 데이터 준비<hr>

In [10]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchinfo import summary

In [11]:
data_file = "../Data/mnist_train.csv"

dataDF = pd.read_csv(data_file, header=None)
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 59.9 MB


In [12]:
## 데이터 텐서 변환
featureDF = dataDF.iloc[:, 1:].values
xTS = torch.tensor(featureDF, dtype=torch.float32) / 255.0

targetDF = dataDF.iloc[:, 0].values
yTS = torch.tensor(targetDF, dtype=torch.long)

print("xTS : ", xTS.shape,xTS.dim, "\nyTS : ", yTS.shape, yTS.dim )


xTS :  torch.Size([10001, 784]) <built-in method dim of Tensor object at 0x000001E1D04CE610> 
yTS :  torch.Size([10001]) <built-in method dim of Tensor object at 0x000001E1B8A39D00>


[2] ann 모델 설계<hr>

In [ ]:
## ==========================================================
##          입력수      퍼셉트론수/출력수       AF
## ==========================================================
## 입력층     784            784          ★Pytorch에는 입력층(클래스) X, 입력 텐서를 입력층으로 간주
## 은닉층     784            128              ReLU
## 은닉층     128            64               ReLU
## 출력층     64             10              softmax
## ==========================================================
## 클래스이름 : MnistModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)  : 층 구성 요소 인스턴스 생성
##            forward(self, x) : 순전파 진행 메서드, ★ x:입력층
## ==========================================================
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hd1_layer = nn.Linear(784, 128)
        self.hd2_layer = nn.Linear(128, 64)
        self.out_layer = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.hd1_layer(x))
        x = F.relu(self.hd2_layer(x))
        x = self.out_layer(x)   # CrossEntropyLoss()을 쓸경우 자동으로 SOFTMAX
        return x


In [14]:
## 모델 객체 생성
model = MnistModel()
summary(model, input_size=(2,784))

Layer (type:depth-idx)                   Output Shape              Param #
MnistModel                               [2, 10]                   --
├─Linear: 1-1                            [2, 128]                  100,480
├─Linear: 1-2                            [2, 64]                   8,256
├─Linear: 1-3                            [2, 10]                   650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.22
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.44
Estimated Total Size (MB): 0.45

[3] 학습 준비 <hr>

In [15]:
## [3-1] 학습 관련 설정값들
EPOCHS = 50
BATCH_SIZE = 200
COUNT = xTS.shape[0] // BATCH_SIZE

In [16]:
## [3-2] 학습 관련 인스턴스들
## -> 모델 인스턴스
model = MnistModel()

## -> 손실 계산 객체
loss_fn = nn.CrossEntropyLoss()

## -> 최적화 인스턴스
adamOpt = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(EPOCHS + 1):
    total_loss = 0.0

    for idx in range(COUNT):
        xb = xTS[idx*BATCH_SIZE:(idx+1)*BATCH_SIZE]
        yb = yTS[idx*BATCH_SIZE:(idx+1)*BATCH_SIZE]

        logits = model(xb) ## 순전파 진행
        loss = loss_fn(logits, yb)

        adamOpt.zero_grad() ## 기울기 초기화
        loss.backward()     ## 역전파 진행
        adamOpt.step()      ## 가중치 업데이트 및 최적화

        total_loss += loss.item()

    if epoch % 5 == 0:
        with torch.no_grad():
            pred = model(xTS).argmax(dim=1)
            acc = (pred == yTS).float().mean().item()
        print(f"[{epoch:03}] loss={total_loss/COUNT:.4f}, acc={acc:.4f}")

[000] loss=1.3773, acc=0.8634
[005] loss=0.2153, acc=0.9477
[010] loss=0.1236, acc=0.9713
[015] loss=0.0756, acc=0.9826
[020] loss=0.0456, acc=0.9905
[025] loss=0.0271, acc=0.9951
[030] loss=0.0173, acc=0.9951
[035] loss=0.0124, acc=0.9888
[040] loss=0.0163, acc=0.9898
[045] loss=0.0121, acc=0.9972
[050] loss=0.0032, acc=0.9991
